## Carga de tablas

Se cargan dos tablas:
- ```df``` que almacena ID, nombre interno y nombre de display de todos los héroes del juego
- ```stats_df```que almacena la información estadística en la que se basará el cálculo de la utilidad

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from tabulate import tabulate
import statistics

In [ ]:
r = requests.get("https://api.stratz.com/api/v1/Hero") 
  
data = r.json()

ids = []
names = []
short_names = []

for key in data:
  ids.append(data[key]['id'])
  names.append(data[key]['displayName'])
  short_names.append(data[key]['shortName'])

df = pd.DataFrame({'ID': ids, 'Name': names, 'ShortName': short_names})
df.head()

In [ ]:
stats_df = pd.read_csv('ratios1.csv')
stats_df.columns = ['ID','name','pro_pick','pro_win','pro_ban']
stats_df['pickrate_weight'] = (stats_df['pro_pick']-stats_df['pro_pick'].min())/(stats_df['pro_pick'].max()-stats_df['pro_pick'].min())

def getpoints(hero):
  win = stats_df[stats_df.name == hero].reset_index(drop=True)['pro_win'][0]
  pick = stats_df[stats_df.name == hero].reset_index(drop=True)['pro_pick'][0]
  peso = stats_df[stats_df.name == hero].reset_index(drop=True)['pickrate_weight'][0]
  return (win/pick - 0.5)*peso*100

## Predicción de modelo

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_auc_score, roc_curve
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
with open("heroEncoder", "rb") as f: 
    enc = pickle.load(f)
with open("labelEncoder", "rb") as f: 
    labEncoder = pickle.load(f)

In [ ]:
def getOneHot(lstHeroes):
  a = None
  flag = True
  for i in lstHeroes:
    if flag: 
      flag = False
      a = enc.transform([[i]]).toarray()
    else:
      a = a + enc.transform([[i]]).toarray()
  return a  

In [ ]:
bst = xgb.Booster({'nthread':4})
bst.load_model("modelo.bin")

In [ ]:
#A Dire le corresponde 0 y a Radiant 1
labEncoder.transform(["Dire","Radiant"])

In [ ]:
def get_model_points(radiant,dire):
  datapoint = np.concatenate((getOneHot(radiant)[0], getOneHot(dire)[0]), axis=0)  
  dmatrix = xgb.DMatrix([datapoint])
  pred_val = bst.predict(dmatrix)[0]
  return pred_val
  #if pred_val - 0.5 < 0: return -1
  #else: return 1

## Definición de clases y funciones
Se definen las siguientes clases:
- ```Game``` clase genérica de la cual hereda el juego
- ```dota_draft``` clase con la implementación de funciones del juego
- ```MCT_Node``` clase necesaria para la implementación de Montecarlo

Así mismo las siguientes funciones:
- ```ucb``` función estadística que calcula el valor según el cual se abren los nodos
- ```monte_carlo_tree_search``` función que ejecuta la búsqueda de Montecarlo

In [ ]:
class Game:

    def actions(self, state):
        """Retorna una lista de movidas permitidas en el estado actual state."""
        raise NotImplementedError

    def result(self, state, move):
        """Retorna el nuevo estado que resulta de hacer una movida move en el estado state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Retorna el valor de utilidad para el jugador player en el estado terminal state."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Retorna True si el estado state es un estado terminal del juego."""
        return not self.actions(state)

    def to_move(self, state):
        """Retorna el jugador que le toca jugar en el presente estado state."""
        return state.to_move

    def display(self, state):
        """Imprime o displaya el state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        state = self.initial
        draft_order = self.draft_order

        for i in range(len(draft_order)):
          if self.terminal_test(state):
            break
          #team = 'RADIANT' if draft_order[i] == 0 else 'DIRE'
          #print("TURNO {}: JUEGA {}".format(i+1, team))
          #print("")
          move = players[draft_order[i]](self, state)
          state = self.result(state, move)
        self.display(state)
        return self.utility(state, self.to_move(self.initial)), (state.radiant, state.dire) #retorna utilidad del 1er jugador al acabar el juego

In [ ]:
from collections import namedtuple
GameState = namedtuple('GameState', 'to_move, utility, radiant, dire, moves, turn') #Un estado es una tupla con nombres de campos (namedtuple)
import random
import itertools
import copy

class dota_draft(Game):
    
    def __init__(self, typeofcomputation='ranking'):
        self.typeofcomputation = typeofcomputation
        moves = [x for x in short_names]
        cols_count = 22
        rows_count = 2
        radiant_draft = []
        dire_draft = []
        radiant_picks = []
        dire_picks = []
        self.radiant_bans = []
        self.dire_bans = []
        # d->1 r->0
        self.draft_order = [0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,1,0,0,1,0,1,0,1,-1]
        self.draft_pick_seq = [0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,1,1,1,0,0,0,0,1,1] # 0=ban 1=pick
        self.initial = GameState(to_move='r', utility=0, radiant=radiant_picks, dire=dire_picks, moves=moves, turn=0)

    def actions(self, state):
        """Movidas legales son todas las posiciones aun sin marcar (el estado almacena las movidas legales)"""
        return state.moves

    def result(self, state, move):
        """Retorna el nuevo estado de hacer la movida move en el estado state ."""
        if move not in state.moves:
            return state  # Si es una movida ilegal retorna sin cambiar el estado
        if self.terminal_test(state):
            return state

        if state.to_move == 'r':
          board = state.radiant.copy()
          other = state.dire.copy()
        else:
          board = state.dire.copy()
          other = state.radiant.copy()

        if self.draft_pick_seq[state.turn] == 1:
          board.append(move)
        else:
          if state.to_move == 'r':
            self.radiant_bans.append(move)
          else:
            self.dire_bans.append(move)
        
        moves = list(state.moves)
        moves.remove(move)

        turno = state.turn + 1
        #state.turn = state.turn + 1
          
        return GameState(to_move=('d' if self.draft_order[turno] == 1 else 'r'),
                         utility=self.compute_utility(board, other, state.to_move),
                         radiant=(board if state.to_move == 'r' else other),
                         dire=(board if state.to_move == 'd' else other), moves=moves, turn=turno)

    def utility(self, state, player):
        """Retorna la utilidad del player en estado terminal state; 1 si ganó, -1 si perdió, 0 empate."""
        return state.utility if player=='r' else -state.utility

    def terminal_test(self, state):
        """Un estado es terminal si hay un ganador o no hay mas movidas posibles."""
        return state.turn >= len(self.draft_pick_seq)

    def display(self, state):
        radiant_picks = state.radiant.copy()
        dire_picks = state.dire.copy()
        #r = np.full(11, '', dtype=np.dtype('U100'))
        #d = np.full(11, '', dtype=np.dtype('U100'))
        
        #print('Radiant picks: ', radiant_picks)
        #print('Dire picks: ', dire_picks)
        #print('Radiant bans: ', self.radiant_bans)
        #print('Dire bans: ', self.dire_bans)

        #for i in range(len(radiant_picks)):
          #r[i] = radiant_picks[i]
        #for i in range(len(dire_picks)):
          #d[i] = dire_picks[i]

        #numpy_data = np.array([r, d])
        #df = pd.DataFrame(data=numpy_data)
        #df.columns =['BAN', 'BAN', 'BAN', 'BAN', 'PICK', 'PICK', 'BAN', 'PICK', 'PICK', 'BAN', 'PICK'] 
        #df.index = ['Radiant','Dire']
        #print(tabulate(df, headers='keys', tablefmt='psql'))

    def compute_utility(self, board, other, player):
      if self.typeofcomputation == 'xgb':
        if (len(board) == 5 and len(other) == 5):
          temp = get_model_points(radiant,dire)
          if player == 'r': return temp
          elif player == 'd': return -temp
        else:
          return 0
      elif self.typeofcomputation == 'ranking':
        if (len(board) == 5 and len(other) == 5):
          temp = 0
          for hero in board:
              temp = temp + getpoints(hero)
          for hero in other:
              temp = temp - getpoints(hero)
          if player == 'r':
            return temp
          elif player == 'd':
            return -temp
        else:
          return 0

In [ ]:
class MCT_Node:
    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state, U=U, N=N)
        self.children = {}
        self.actions = None

In [ ]:
def ucb(n, C=1.4):
    if n.N == 0:
        return np.inf    
    else:
        return (n.U / n.N) + C * np.sqrt(np.log(n.parent.N) / n.N)

In [ ]:
def monte_carlo_tree_search(state, game, N=100):
    def select(n):
        """select a leaf node in the tree"""
        if n.children:
            return select(max(n.children.keys(), key=ucb))
        else:
            return n

    def expand(n):
        """expand the leaf node by adding all its children states"""
        if not n.children and not game.terminal_test(n.state):
            n.children = {MCT_Node(state=game.result(n.state, action), parent=n): action
                          for action in game.actions(n.state)}
        return select(n)

    def simulate(game, state):
        """simulate the utility of current state by random picking a step"""
        player = game.to_move(state)
        #print(player)
        #print(state)
        while not game.terminal_test(state):
            #print(state)
            action = random.choice(list(game.actions(state)))
            state = game.result(state, action)
        v = game.utility(state, player)
        return -v

    def backprop(n, utility):
        """passing the utility back to all parent nodes"""
        if utility > 0:
            n.U += utility
        # if utility == 0:
        #     n.U += 0.5
        n.N += 1
        if n.parent:
            backprop(n.parent, -utility)

    root = MCT_Node(state=state)

    for i in range(N):
        #print('iteracion ', i)
        leaf = select(root)
        child = expand(leaf)
        result = simulate(game, child.state)
        backprop(child, result)

    max_state = max(root.children, key=lambda p: p.N)

    return root.children.get(max_state)

## Creación de jugadores humanos
Se crean los siguientes jugadores:
- ```human_player``` para un jugador humano
- ```mcts_player``` para la búsqueda de Montecarlo
- ```random_player``` para un jugador que elige aleatoriamente

In [ ]:
def human_player(game, state):
    game.display(state)
    print("Movidas disponibles: {}".format(game.actions(state)))
    print("")
    move_string = input('Elija un héroe ')
  
    while move_string not in game.actions(state):
      move_string = input('Elija un héroe ')

    print("")
    return move_string

def mcts_player(game, state):
    return monte_carlo_tree_search(state, game)

def random_player(game,state):
    return random.choice(game.actions(state))

## Creación de funciones para obtener métrica
Estamos considerando como dos métricas  el porcentaje de ventaja predicho por el sitio web NeuralDota. Por ello creamos las siguientes funciones:
- ```getNDPrediction``` para extraer la ventaja de un draft o de otro (valores positivos para Radiant y negativos para Dire)

In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import time
import re

def getNDPrediction(radiant,dire):

    def fancynames(team):
        fancyteam = []
        for name in team:
          fancyname = df[df['ShortName'] == name]['Name'].to_string(index=False)
          fancyteam.append(fancyname[1:])
        return fancyteam

    driver = Chrome('chromedriver.exe')
    link = 'https://neuraldota.com/team-neural-network'
    driver.get(link)
    driver.find_element_by_css_selector(".mat-h1:nth-child(1)").click()
    driver.find_elements_by_xpath("//*[contains(text(), ' Pick as Radiant ')]")[0].click()
    for i in range(5):
        nombre = fancynames(radiant)[i]
        time.sleep(2)
        actions = ActionChains(driver)
        actions.send_keys(nombre)
        actions.send_keys(Keys.ENTER)
        actions.perform()
        nombre = fancynames(dire)[i]
        time.sleep(2)
        actions = ActionChains(driver)
        actions.send_keys(nombre)
        actions.send_keys(Keys.ENTER)
        actions.perform()
    text = driver.find_elements_by_xpath("//*[contains(text(), ' All games ')]")[0].find_elements_by_css_selector("*")[0].text
    driver.quit()
    if re.search('Radiant',text):
        return float(re.findall("\+(.*?)%\)",text)[0])
    elif re.search('Dire',text):
        return -float(re.findall("\+(.*?)%\)",text)[0])



In [ ]:
dota = dota_draft()
prediction, (radiant, dire)= dota.play_game(random_player, random_player)

In [ ]:
n_tests = 20
for scoring in ['ranking','xgb']:
    coincidencias = 0
    diferencias = []
    for _ in range(n_tests):
        dota = dota_draft(typeofcomputation = scoring)
        prediction, (radiant, dire) = dota.play_game(mcts_player, random_player)
        ndprediction = getNDPrediction(radiant, dire)
        if (ndprediction*prediction > 0):
            coincidencias = coincidencias + 1
        diferencia = abs(ndprediction-prediction)
        diferencias.append(diferencia)
        print("Coincidencia?",bool(coincidencias),". Diferencia entre preds:",diferencia)

    print('Número de veces que el resultado del draft coincide con el cálculo de ND:',coincidencias)
    print('Desviación estándar entre resultado del draft y cálculo de ND:',statistics.stdev(diferencias))

## Misceláneo
Ejemplo de ejecución independiente. Se puede cambiar los jugadores por alguna de las siguientes opciones:
- ```mcts_player```: Utiliza el agente Monte Carlo
- ```random_player```: Prohibe y elige héroes al azar
- ```human_player```: Humano elige héroes

In [ ]:
%%time
dota = dota_draft(typeofcomputation = 'ranking')
prediction, (radiant, dire) = dota.play_game(mcts_player, random_player)
print(prediction,radiant,dire)

In [ ]:
%%time
dota = dota_draft(typeofcomputation = 'xgb')
prediction, (radiant, dire) = dota.play_game(mcts_player, random_player)
print(prediction,radiant,dire)